<a href="https://colab.research.google.com/github/3B032013/2025_DL_hws/blob/main/hw5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. 將存好的模型和訓練權重讀回來用

In [145]:
from google.colab import drive

drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [146]:
from tensorflow.keras.models import model_from_json
model = model_from_json(open('imdb_model_architecture.json').read())
model.load_weights('imdb_model.weights.h5')

In [147]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [148]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 32)        │        64,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        82,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 146,561 (572.50 KB)

 Trainable params: 146,561 (572.50 KB)

 Non-trainable params: 0 (0.00 B)

# 2. 載入自行建立的tokenizer

In [149]:
import pickle
f = open('SaveModel/imdb_tokenizer_2000.pkl', 'rb')
tokens = pickle.load(f)
f.close()

In [150]:
from tensorflow.keras.preprocessing import sequence

3. 輸入實際由IMDb取得的影評

In [151]:
input_text= '''
Sure, I'm a huge film snob who (on the surface) only likes artsy-fartsy foreign films from before the 60's, but that hasn't stopped me from loving Disney's Beauty & The Beast; in fact, it's probably my favorite American animated film and is easily Disney's finest work. It's beautiful, it's breathtaking, it's warm, it's hilarious, it's captivating, and, in Disney fashion, it's magical. When I learned that Disney would be remaking their classic films, B&TB was undeniably the best wrapped package. How could they go wrong?
Oh man, they went wrong.
First thing's first: this film is so flat. The directing was dull and uninteresting throughout the entire film and it honestly felt like one of the Twilight sequels...and then I looked it up and found out that, yes, director Bill Condon was the man behind Breaking Dawn parts 1 & 2. Every shot looks bored and uninterested, which contrasts heavily with the original animated film that was constantly popping with vibrancy. The script too is boring because it's almost a complete remake of the original, though I guess most people won't mind that.
Next: the CGI is horrid. Although I didn't care for The Jungle Book from last year, I could at least admit that the CGI was breathtaking. The same cant be said for this film. Characters like Lumière, Cogsworth, Mrs Potts, and most of the cursed appliances have very strange, lifeless faces that are pretty off putting to be looking at for such a long time. All of the sets too look artificial and fake, especially the town towards the beginning. However, the biggest offender is easily and infuriatingly the character that mattered most: The Beast. The CGI on the Beast's face is so distracting that it completely takes you out of the film. His eyes are completely devoid of soul, and his mouth is a gaping video game black hole of fiction. Klaus Kinski looked much better in the Faerie Tale Theatre episode of Beauty & The Beast, and that was a 1984 TV show episode. But do you know why it looked better? Because it was an actual face with actual eyes, not some video game computerized synthetic monstrosity. When will studios learn that practical effects will always top CGI?
Finally: wasted casting. Emma Watson is beautiful, but she's no Belle. She is completely devoid of the warmth and humanity that made the animated Belle so beloved. Instead, she is cold and heartless throughout most of the film. Kevin Kline is 100% wasted and does nothing except look old. Ian McKellan, Ewan McGregor, Emma Thompson, and even Dan Stevens as the Beast are very expendable and could've been played by anyone else. The only good characters are Gaston and LeFou, mostly because they are fun and played by actors who breathe new life into their original shapes. If anything, this film should've been about Gaston and LeFou, but that would never happen because that would mean Disney couldn't cater to blind nostalgic 90's kids.
Overall, this film is a complete bore. It could've been better if even the special effects were good, but the CGI in particular is horrendous. I'm all for Disney remaking their nostalgia- catering 90's films, but they need to be interesting. This film, sadly, is not. Even the Christmas sequel is better than this film because it's at least something.
'''


# 4. 轉換影評文字串為數字串，並截長或補短，使數字串長度為100

In [152]:
input_seq = tokens.texts_to_sequences([input_text])
pad_input_seq  = sequence.pad_sequences(input_seq , maxlen=100)
pad_input_seq

array([[   2,   73,  252,   30,  255,  331,    1,   60,   48,  101,   22,
           2,  664,   84,   32,   22,  249,    2,  252,   30,  152,   33,
         157,  109,   79,   64,  200,   44,  228,   10,   18,   73,   40,
           2,   17,   11,   58,  111,  589,   84,   11,   58,  377,  910,
         422,    5,  359,  443,   10,   18,    6,    3,  596,    8,   73,
         124,   44,   56,    1,  314,  297,   67,   48,   17,    1, 1681,
           7,  839,    6,  142,   28,   14,  910,   64,  104,   17,   32,
         355,    5,   25,  217,   10,   18, 1032,    6,   20,   56,    1,
         977,  750,    6,  124,   70,   10,   18,   84,   41,   29,  218,
         138]], dtype=int32)

In [153]:
# 測試將影評數字串還原為文字串
pad_input_text = tokens.sequences_to_texts(pad_input_seq)
pad_input_text

["and been played by anyone else the only good characters are and mostly because they are fun and played by actors who new life into their original if anything this film been about and but that would never happen because that would mean disney couldn't to kids overall this film is a complete it been better if even the special effects were good but the cgi in particular is i'm all for disney their films but they need to be interesting this film sadly is not even the christmas sequel is better than this film because it's at least something"]

# 5. 輸入影評(數字串，長度為100)進行情意預測

In [154]:
predict_result=model.predict(pad_input_seq)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step


In [155]:
predict_result

array([[0.00316081]], dtype=float32)

In [156]:
predict_result[0][0]

np.float32(0.003160808)

In [157]:
SentimentDict={1:'正面的',0:'負面的'}

In [158]:
SentimentDict[round(predict_result[0][0])]

'負面的'

# 6. 建立predict_review函數：

In [159]:
def predict_review(input_text):
  input_seq = tokens.texts_to_sequences([input_text])
  pad_input_seq  = sequence.pad_sequences(input_seq , maxlen=100)
  predict_result=model.predict(pad_input_seq)
  SentimentDict={1:'正面的',0:'負面的'}
  result = SentimentDict[round(predict_result[0][0])]
  return result

# 7. 影評情意預測

## 從IMDB網站找了三則影評如下

## 第一則 網站上為 正評10/10

In [160]:
# 網址：https://www.imdb.com/title/tt0068646/review/rw4059579/?ref_=tturv_3
# 此則為正評 10/10分

input_text_1 = '''
This isn't just a beautifully crafted gangster film. Or an outstanding family portrait, for that matter. An amazing period piece. A character study. A lesson in filmmaking and an inspiration to generations of actors, directors, screenwriters and producers. For me, this is more: this is the definitive film. 10 stars out of 10.
'''

In [161]:
# 預測結果
predict_review(input_text_1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


'正面的'

## 第二則 網站上為 負評1/10

In [162]:
# 網址：https://www.imdb.com/title/tt0068646/review/rw5885143/?ref_=tturv_1
# 此則為負評 1/10分

input_text_2 = '''
Over the years, I had heard a lot about this movie but, for some reason, had never seen it. About two years ago, I found myself with a free evening and began searching for a movie to watch online. I ran across The Godfather and thought all right, I'll finally get to see this masterpiece!
Although I began watching it with high expectations, they were gone within about 20 minutes. I found it to be boring to the extreme and almost turned it off on several occasions, but I decided to stick with it in the hope that it would improve. Unfortunately, it didn't. By the time that it had ended, I was actually mad that I had wasted over two hours of my life watching it.
I realize that I'm in the extreme minority, but I absolutely did not like this movie at all.
'''

In [163]:
# 預測結果
predict_review(input_text_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


'負面的'

## 第三則 網站上為 負評1/10

In [164]:
# 第三則 網址：https://www.imdb.com/title/tt0068646/review/rw0123284/?ref_=tturv_10
# 此則為負評 1/10分

input_text_3 = '''
Let me just start by saying that I am 50% Italian. I have nothing against Mafia movies. In fact, I have nothing against ANY type of movie. But, to say that this is the best movie of all time is simply insane.
I saw absolutely nothing special. I kind of enjoyed the performances of Al Pacino, Marlon Brando, and James Caan... but besides that, there is not one really exceptional feature in sight. And the fact that it is one hour too long only brought down my enjoyment. It's not the worst film of all time... but not the best either.
'''

In [165]:
# 預測結果
predict_review(input_text_3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


'負面的'

# 8. 建立Gradio 直接輸入影評資料 即可直接回答此評論是正面的或負面的

In [166]:
!pip install gradio
import gradio as gr
from keras.preprocessing import sequence

# 建立 Gradio 介面
demo = gr.Interface(
    fn=predict_review,
    inputs=gr.Textbox(lines=5, placeholder="請輸入一段電影評論..."),
    outputs="text",
    title="IMDB 電影評論情緒判斷",
    description="輸入一段影評，我會判斷它是正面還是負面!"
)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a58f32fe72e25a09a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
